## Introduction
This notebook is intended as a guide to my current toy project of making a
reinforcement-learning based chess-playing program. It's inspired by alphazero, 
but it's more just a tool for me to learn a bit about reinforcement learning than anything else. In other words: Don't take it too seriously.
    
The program is built in multiple layers.   

* The first layer is the basic game mechanics of chess: Move around on the board, find legal moves, print out board positions and moves etc. Chess is implemented, but there are some elements missing. In particular there is no rooking, no en passant, and no draws forced from repeated moves.   These can be added, but I felt that the subset of chess currently implemented is fine as a "minimum viable chess", for the purposes of this project. 

* The second layer is about gameplay.  How to play against an opponent, how to run tournaments and score the performance of the participants in the tournaments.   At this time the only tournaments are between two players, but they do play a number of matches 

* The third layer is about reinforcement learning.  In particular deep reinforcement learning.    It is by far the least mature of the three layers, but it is complete to the extent that it is possible to put run tournaments between players that are defined by artificual neural networks implemented using Flux.jl.

This notebook will  present each of these layers and how to access them as a developer.   The first thing to do is to include the "chez.jl" file.  This will load all the necessary libraries, and run some unit tests etc.

Some of these unit tests involve playing a short turnament, so don't be alarmed if it says so.

    
    

In [4]:
include("chez.jl")

8♜♞♝♛♚♝♞♜8
7♟♟♟♟♟♟♟♟7
6        6
5        5
4        4
3        3
2♙♙♙♙♙♙♙♙2
1♖♘♗♕♔♗♘♖1
Playing tournament
Tournament played
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


run_big_tournament (generic function with 1 method)

## Chess pieces



With that in place, let's look at the chess pieces and the chess board.

There is a type for every color and every type of piece (rook, pawn, etc.)  It all comes together in the struct ChessPiece found in the pieces.jl file.

     struct ChessPiece
       color:: Color
       piecetype:: PieceType
       printrep:: String
       unicode:: String
     end

The "printrep" is a single character written description of the piece. Black pieces are given upper case names (like "P" for black pawns).   The "unicode" is a unicode chess symbol character, so for a black pawn it would be "♟".

All the pieces are also given symbolic names, so it's possible to refer to individual pieces by the variable that represents them:



    bp  = ChessPiece(black, pawn,   "P", "♟");
    br  = ChessPiece(black, rook,   "R", "♜");
    bk  = ChessPiece(black, knight, "G", "♞")
    bb  = ChessPiece(black, bishop, "B", "♝");
    bq  = ChessPiece(black, queen,  "Q", "♛");
    bki = ChessPiece(black, king,   "K", "♚");

    wp  = ChessPiece(white, pawn,   "p", "♙");
    wr  = ChessPiece(white, rook,   "r", "♖");
    wk  = ChessPiece(white, knight, "g", "♘");
    wb  = ChessPiece(white, bishop, "b", "♗");
    wq  = ChessPiece(white, queen,  "q", "♕");
    wki = ChessPiece(white, king,   "k", "♔");

    bs = ChessPiece(transparent, blank,  " ",  " ");
    
That's pretty much it


In [6]:
 # a black pawn will print as a string with a capital P.
bp 

"P"

## Chessboard

Iin the file "chessboard.jl" we introduce a data structure  # a black pawn will print as a string with a capital P. for holding chessboards and game positions. It's implemented as an array of chess pieces. 


We then produce a constant we call startingBoard, which contains the fully populated chessboard, as it appears in standard chess before the first move is performed.

There are some structures for describing coordinates and positioning of pieces, but not much more in chessboard.jl.


# Constructing an initial chessboard

    startingBoardArray = [
      wr wk wb wq wki wb wk wr;
      wp wp wp wp wp  wp wp wp;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bp bp bp bp bp  bp bp bp;
      br bk bb bq bki bb bk br;
    ];

    startingBoard = ChessBoard(startingBoardArray)
    
    
 The startingBoardArray is simply an of chess pieces, and we print it using the string representation of the pieces.


In [10]:
startingBoardArray


8×8 Array{ChessPiece,2}:
 "r"  "g"  "b"  "q"  "k"  "b"  "g"  "r"
 "p"  "p"  "p"  "p"  "p"  "p"  "p"  "p"
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 "P"  "P"  "P"  "P"  "P"  "P"  "P"  "P"
 "R"  "G"  "B"  "Q"  "K"  "B"  "G"  "R"

The startingBoard object however, we chose to print using the unicode "chess" characters.  This is what we will ordinarily use, since it (according to my taste) looks nicer and more compact.

In [9]:
startingBoard

8♜♞♝♛♚♝♞♜8
7♟♟♟♟♟♟♟♟7
6        6
5        5
4        4
3        3
2♙♙♙♙♙♙♙♙2
1♖♘♗♕♔♗♘♖1


## Moving around

    struct Move
        start:: Coord
        destination:: Coord
        capture:: Bool
        startPiece:: ChessPiece
        destinationPiece:: ChessPiece
    end

